참고문헌 : Pytel 외 저, 이주성 외 역, 재료역학, 2판, 한티미디어, 2013.<br>Ref: Pytel, Kiusalaas, Sharma, Mechanics of Materials, 2nd Ed., Cengage Learning, 2013.

`python` 기능을 확장해 주는 `module`을 불러 들임 (일부 기능만 사용될 수도 있음)<br>
Bring in `module`'s that would expand features of `python`. (This file may use just some of them.)

In [ ]:
import numpy as np  # 배열, 행렬 관련 기능
import numpy.linalg as na  # 선형대수 (벡터, 행렬) 관련 기능
import matplotlib.pyplot as plt  # 그래프 관련 기능
import scipy.integrate as si  # 적분 관련 기능
import sympy as sy  # 기호 연산 기능
import IPython.display as disp  # 웹페이지 표시 기능
sy.init_printing()  # 기호 연산 결과 표시 기능 준비

## 예제 03.007

### 문제에서 주어진 변수

다음과 같은 변수들이 문제에서 주어졌다.

In [ ]:
L_m, t_1_m, t_2_m, G_al_Pa, r_m = sy.symbols('L[m], t_{1}[m], t_{2}[m], G_{al}[Pa], r[m]')

mm 단위 변수도 준비해 둔다.

In [ ]:
t_1_mm = 1e-3 * t_1_m
t_2_mm = 1e-3 * t_2_m
r_mm = 1e-3 * r_m

변수값을 나중에 사용하기 위해 `dict` 형태로 저장해 둔다.

In [ ]:
subs_dict = {L_m: 1.2,  # 관의 길이
             t_1_mm: 2,  # 관의 벽 두께 1
             t_2_mm: 3,  # 관의 벽 두께 2
             G_al_Pa: 28e9,  # 알루미늄의 전단탄성계수
             r_mm: 25,  # 관의 반지름
            }
subs_dict[t_1_m] = subs_dict[t_1_mm] * 1e-3  # 각관의 벽 두께 1
subs_dict[t_2_m] = subs_dict[t_2_mm] * 1e-3  # 각관의 벽 두께 2
subs_dict[r_m] = subs_dict[r_mm] * 1e-3  # 관의 반지름
subs_dict

### 1) 최대 전단응력 40MPa 를 일으키는 토크

In [ ]:
T_Nm = sy.symbols('T[Nm]')

In [ ]:
tau_max_al_Pa = sy.symbols('tau_{max}_{al}[Pa]')
subs_dict[tau_max_al_Pa] = 40e6  # 알루미늄의 최대전단응력

최대 전단응력은 두께가 가장 얇은 곳에서 발생함
$$\tau=\frac{T}{2A_0t}$$

전단응력을 계산하기 위해 중심선 안 면적을 지정

In [ ]:
A_0_m2 = (sy.pi * r_m**2)/2
A_0_m2

두께가 $t_1$인 부분의 전단응력

In [ ]:
tau_1_Pa = T_Nm/(2 *A_0_m2 * t_1_m)
disp.display(tau_1_Pa)

두께가 $t_2$인 부분의 전단응력

In [ ]:
tau_2_Pa = T_Nm/(2 *A_0_m2 * t_2_m)
disp.display(tau_2_Pa)

최대전단응력을 일으키는 토크를 구하면 각각 다음과 같다.

#### $t_1$인 부분에 최대 전단 응력을 일으키는 토크

In [ ]:
T_max_t1_Nm_sol_list = sy.solve(T_Nm/(2 *A_0_m2 * t_1_m) - tau_max_al_Pa, T_Nm)
T_max_t1_Nm_sol_list

변수 값을 대입하면 다음과 같다 [Nm]

In [ ]:
T_max_t1_Nm = T_max_t1_Nm_sol_list[0].subs(subs_dict)
T_max_t1_Nm

이때 $t_2$ 인 부분에 발생하는 응력 [MPa]

In [ ]:
tau_2_Pa.subs(T_Nm, T_max_t1_Nm).subs(subs_dict) * 1e-6

이 값은 허용 최대 전단 응력을 넘지 않는다.

#### $t_2$인 부분에 최대 전단 응력을 일으키는 토크

In [ ]:
T_max_t2_Nm = sy.solve(T_Nm/(2 *A_0_m2 * t_2_m) - tau_max_al_Pa, T_Nm)[0].subs(subs_dict)
T_max_t2_Nm

이 때 $t_1$인 부분에 발생하는 응력 [MPa]

In [ ]:
tau_1_Pa.subs(T_Nm, T_max_t2_Nm).subs(subs_dict) * 1e-6

이 값은 허용 최대 전단 응력을 넘는 값이므로 "$t_1$인 부분" 의 경우를 채택한다.

### 2) 그 토크가 가해질 때의 비틀림 각

$$\theta = \frac{TL}{4GA_0^2}\oint \frac{1}{t}ds$$

반원부분의 길이 $S_1$

In [ ]:
S_1_m = sy.pi * r_m
S_1_m

직선부분의 길이 $S_2$

In [ ]:
S_2_m = 2 * r_m
S_2_m

$$ 
t(s) = t_1, 0 \le s \le \pi r\\
t(s) = t_2, \pi r \le s \le \pi r + 2 r
$$

적분부분

$$
\oint \frac{1}{t} ds = \int_{s=0}^{s=\pi r} \frac{1}{t_1}ds+\int_{s=\pi r}^{s=\pi r + 2r} \frac{1}{t_2}ds
$$

$t_1$, $t_2$는 $s$ 가 변하더라도 상수이다. 

$$
\oint \frac{1}{t} ds = \frac{1}{t_1}\int_{s=0}^{s=\pi r} ds+\frac{1}{t_2}\int_{s=\pi r}^{s=\pi r + 2r} ds
$$

$$
\oint \frac{1}{t} ds = \frac{1}{t_1}\left[s\right]_{s=0}^{s=\pi r} +\frac{1}{t_2}\left[s\right]_{s=\pi r}^{s=\pi r + 2r}
$$

$$
\oint \frac{1}{t} ds = \frac{1}{t_1}\pi r +\frac{1}{t_2}2r
$$

In [ ]:
int_part = S_1_m/t_1_m + S_2_m/t_2_m
disp.display(int_part)
float(int_part.subs(subs_dict))

비틀림 각 변위

In [ ]:
theta_rad = T_Nm * L_m / (4 * G_al_Pa * A_0_m2**2) * int_part
theta_rad

In [ ]:
theta_rad_val = float(theta_rad.subs(T_Nm, T_max_t1_Nm).subs(subs_dict))
disp.display(disp.Math('\\theta = %g (rad)' % theta_rad_val))
disp.display(disp.Math('\\theta = %g (deg)' % np.rad2deg(theta_rad_val)))